In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F#i assume this is the functional api

In [ ]:
#Lets define our neural network
class Perceptron(nn.Module):

    def __init__(self):
        super(Perceptron, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(2, 2)
    #we define this function to tell the network how to pass the input through the network.
    #the backwards function will automatically be defines using autograd
    def forward(self, x):
        x = (self.fc1(x))
        return x


In [ ]:
classifier = Perceptron()
print(classifier)

In [ ]:
xor = pd.DataFrame({'X1':[0.,0.,1.,1.], 'X2':[0.,1.,0.,1.], 'y':[0,1,1,0]})
xor

In [ ]:
#Lets load in our datasets:
import torch.utils.data as data_utils
#Why do we have to use from numpy?
features = torch.from_numpy(xor.iloc[:,:2].values).float()
targets = torch.from_numpy(xor.iloc[:,2].values).long()
train = data_utils.TensorDataset(features, targets)
trainloader = data_utils.DataLoader(train, batch_size=1, shuffle=True)

In [ ]:
y_hat.shape

In [ ]:
import torch.optim as optim
epochs = 1
criterion = F.cross_entropy#loss function
optimizer = optim.SGD(classifier.parameters(), lr=0.001)
for epoch in range(100):
    print("Epoch: {}".format(epoch))
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        x, labels = data
        # zero the parameter gradients, we do not want to keep adding to the gradient after each minibatch
        #what if we didnt do this?
        optimizer.zero_grad()
        
        #forward
        y_hat = classifier(x)
        loss = criterion(y_hat, labels)#get the loss between the target and output, gets us our error
        running_loss += loss
        #backwards
        loss.backward()#back prop
        
        #Gradient Descent
        optimizer.step()#perform stochastic gradient desent.
    print('loss: {}'.format(running_loss/len(train)))

In [ ]:
y_hat

In [ ]:
# Visualising the Testing set results
from matplotlib.colors import ListedColormap
X_set, y_set = xor.iloc[:,:2].values, xor.iloc[:,2].values
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
input_ = torch.tensor(np.array([X1.ravel(), X2.ravel()]).T).float()
plt.contourf(X1, X2, torch.argmax(classifier(input_), dim=1).detach().numpy().reshape(X1.shape),
             alpha = 0.5, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('XOR problem 3 classes - Iris Dataset')
plt.xlabel('petal-length')
plt.ylabel('petal-width')
plt.legend()
plt.show()